In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .appName("HELK Reader") \
    .master("spark://helk-spark-master:7077") \
    .enableHiveSupport() \
    .getOrCreate()

In [3]:
es_reader = (spark.read
    .format("org.elasticsearch.spark.sql")
    .option("inferSchema", "true")
    .option("es.read.field.as.array.include", "tags")
    .option("es.nodes","helk-elasticsearch:9200")
    .option("es.net.http.auth.user","elastic")
)

In [4]:
sysmon_df = es_reader.load("logs-endpoint-winevent-sysmon-*/")

In [5]:
sysmon_df.createOrReplaceTempView("sysmon_events")

In [44]:
Hook_Injection = spark.sql(
'''
SELECT 
    a.process_name,
    a.process_guid,
    b.process_guid,
    c.process_guid,
    d.process_guid
FROM sysmon_events b
JOIN sysmon_events a
    ON a.process_target_guid = b.process_guid
    AND a.event_id = 10
    AND b.process_integrity_level = "High"
    AND (a.process_granted_access & 40) == 40
    AND b.process_parent_name = a.process_name
JOIN sysmon_events c
    ON b.process_guid = c.process_guid
    AND c.event_id = 7
    AND c.module_loaded = b.process_path
    AND c.process_guid = a.process_target_guid
    AND c.module_loaded = a.process_target_path
JOIN sysmon_events d
    ON d.process_name = b.process_parent_name
    AND d.event_id = 13
WHERE
     b.event_id = 1
     AND b.process_name = "notepad.exe"
'''
).show(100,False)

+--------------+------------------------------------+------------------------------------+------------------------------------+------------------------------------+
|process_name  |process_guid                        |process_guid                        |process_guid                        |process_guid                        |
+--------------+------------------------------------+------------------------------------+------------------------------------+------------------------------------+
|injectproc.exe|9d556bd4-1442-5e1d-0000-0010ce0ff801|9d556bd4-1442-5e1d-0000-00109812f801|9d556bd4-1442-5e1d-0000-00109812f801|9d556bd4-1442-5e1d-0000-0010ce0ff801|
+--------------+------------------------------------+------------------------------------+------------------------------------+------------------------------------+



In [20]:
AND (process_granted_access & 5184) == 5184 -- 5184 is decimal for 0x1440. The minimal privileges you need to access process handle
    AND a.process_granted_access = "2047999"

SyntaxError: invalid syntax (<ipython-input-20-29f0fe263280>, line 1)

In [26]:
2047999 & 40 == 40

True